In [1]:
!pip install optree


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import json
import sys
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam



In [3]:
tf.__version__

'2.15.0'

In [4]:
try:
    # Attempt to connect to a TPU
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Automatically detects the TPU
    print('Device:', tpu.master())  # Display the TPU address
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)  # Updated to use the non-experimental version
    DEVICE = "TPU"
    print("Running on TPU:", tpu.master())
except ValueError as e:
    # If a TPU is not found, fall back to default strategy (could be CPUs, one GPU or multiple GPUs)
    print("TPU not found, falling back to default strategy (CPU/GPU)")
    print("Error:", e)
    DEVICE = "notTPU"
    strategy = tf.distribute.get_strategy()  # Get the default distribution strategy
    print("Default strategy type:", type(strategy).__name__)



Device: 
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2024-04-13 13:41:07.165502: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:41:07.165613: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:41:07.165696: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:41:07.165784: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:41:07.165849: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:41:07.166071: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:41:07.166163: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this 

INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [5]:
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print('Device:', tpu.master())
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
#     DEVICE = "TPU"
# except:
#     DEVICE = "notTPU"
#     strategy = tf.distribute.get_strategy()

# AUTOTUNE = tf.data.experimental.AUTOTUNE
# GCS_PATH = KaggleDatasets().get_gcs_path(f'cassava-leaf-disease-classification')
REPLICAS =  strategy.num_replicas_in_sync
BATCH_SIZE = 32 * strategy.num_replicas_in_sync
# FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train_tfrecords/*.tfrec')

LR = 2e-6 * REPLICAS
EPOCHS = 5

# img_size = (512, 512)
# default_img_size = (512, 512) 
print(DEVICE)

TPU


In [6]:
# Declare directories
train_dir = "/kaggle/input/cassava-leaf-disease-classification/train_images/"
label_dir = "/kaggle/input/cassava-leaf-disease-classification/train.csv"
idx_to_str_dir = "/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json"

# Loading Data

In [7]:
# Load the labels
labels = pd.read_csv(label_dir)

# Load the mapping from numeric labels to disease names
with open(idx_to_str_dir) as file:
    label_map = json.load(file)

# Add a full path to the image filenames
labels['image_id'] = train_dir + labels['image_id']


## Split Data into Training and Validation Sets

In [8]:

train, valid = train_test_split(labels, test_size=0.2, random_state=42)


In [9]:
print(train.head())  # Check the first few rows of the training data
print(train.dtypes)  # Check the data types of the columns


                                                image_id  label
14618  /kaggle/input/cassava-leaf-disease-classificat...      3
13344  /kaggle/input/cassava-leaf-disease-classificat...      3
11327  /kaggle/input/cassava-leaf-disease-classificat...      3
8757   /kaggle/input/cassava-leaf-disease-classificat...      4
4696   /kaggle/input/cassava-leaf-disease-classificat...      3
image_id    object
label        int64
dtype: object


## Create Data ImageDataGenerators

In [10]:
# Image data augmentation setup
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Make sure labels are strings if using class_mode="categorical"
train['label'] = train['label'].astype(str)
valid['label'] = valid['label'].astype(str)

# Setting up data generators
train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col='image_id',
    y_col='label',
    target_size=(456, 456),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = valid_datagen.flow_from_dataframe(
    valid,
    x_col='image_id',
    y_col='label',
    target_size=(456, 456),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 17117 validated image filenames belonging to 5 classes.
Found 4280 validated image filenames belonging to 5 classes.


## Set up EfficientNetB5 Model

In [11]:
# EfficientNetB5 setup
with strategy.scope():
    # Initialize the base model with pretrained weights and without the top layer
    base_model = EfficientNetB5(include_top=False, weights='imagenet', input_shape=(456, 456, 3))
    base_model.trainable = False  # Freeze the base model initially

    # Construct the full model by appending custom layers on top
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(1024, activation='relu'),
        Dense(len(label_map), activation='softmax')
    ])

    # Compile the model with initial settings
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Setup callbacks
    checkpoint_cb = ModelCheckpoint("best_model.keras", save_best_only=True, monitor='val_accuracy')
    early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss')

    # Initial training (transfer learning)
    history = model.fit(
        train_generator,
        epochs=5,
        validation_data=validation_generator,
        verbose=1,
        callbacks=[checkpoint_cb, early_stopping_cb]
    )

    # Unfreeze the base model for fine-tuning
    base_model.trainable = True

    # Re-compile the model with a lower learning rate for fine-tuning
    model.compile(optimizer=Adam(LR), loss='categorical_crossentropy', metrics=['accuracy'])

    # Fine-tuning the model
    history_fine = model.fit(
        train_generator,
        epochs=5,
        validation_data=validation_generator,
        verbose=1,
        callbacks=[checkpoint_cb, early_stopping_cb]  # Reuse the same callbacks
    )


2024-04-13 13:41:26.040656: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
I0000 00:00:1713015701.029322    8003 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1/5


2024-04-13 13:42:04.323628: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:42:04.348106: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:42:04.371027: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:42:04.393414: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:42:04.416331: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:42:04.440236: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:42:04.464125: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this 

 1/67 ━━━━━━━━━━━━━━━━━━━━ 2:34:24 140s/step - accuracy: 0.2188 - loss: 12.5729

2024-04-13 13:44:23.913470: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:44:23.917620: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:44:23.917742: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:44:23.917819: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:44:23.918485: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:44:23.919159: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-13 13:44:23.920104: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this 

...

67/67 ━━━━━━━━━━━━━━━━━━━━ 906s 12s/step - accuracy: 4.4396 - loss: 10.5044 - val_accuracy: 0.6318 - val_loss: 1.1780
Epoch 2/5
67/67 ━━━━━━━━━━━━━━━━━━━━ 830s 11s/step - accuracy: 4.8821 - loss: 9.5667 - val_accuracy: 0.6206 - val_loss: 1.1918
Epoch 3/5
67/67 ━━━━━━━━━━━━━━━━━━━━ 843s 11s/step - accuracy: 4.8209 - loss: 9.5322 - val_accuracy: 0.6243 - val_loss: 1.1700
Epoch 4/5
67/67 ━━━━━━━━━━━━━━━━━━━━ 866s 12s/step - accuracy: 4.7902 - loss: 9.5854 - val_accuracy: 0.6411 - val_loss: 1.1354
Epoch 5/5
67/67 ━━━━━━━━━━━━━━━━━━━━ 864s 12s/step - accuracy: 4.8485 - loss: 9.4327 - val_accuracy: 0.5963 - val_loss: 1.2050
Epoch 1/5


2024-04-13 14:57:37.536208: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.


67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 2.5477 - loss: 13.9583 

2024-04-13 15:13:39.063155: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignVariableOp.


67/67 ━━━━━━━━━━━━━━━━━━━━ 1243s 13s/step - accuracy: 2.5172 - loss: 13.7749 - val_accuracy: 0.6579 - val_loss: 1.2344
Epoch 2/5
67/67 ━━━━━━━━━━━━━━━━━━━━ 870s 12s/step - accuracy: 4.9026 - loss: 8.6465 - val_accuracy: 0.6019 - val_loss: 1.2854
Epoch 3/5
67/67 ━━━━━━━━━━━━━━━━━━━━ 858s 12s/step - accuracy: 5.2023 - loss: 7.2991 - val_accuracy: 0.6019 - val_loss: 1.2599
Epoch 4/5
67/67 ━━━━━━━━━━━━━━━━━━━━ 863s 12s/step - accuracy: 5.5095 - loss: 6.5139 - val_accuracy: 0.6168 - val_loss: 1.2393
Epoch 5/5
67/67 ━━━━━━━━━━━━━━━━━━━━ 867s 12s/step - accuracy: 5.8081 - loss: 5.9394 - val_accuracy: 0.6056 - val_loss: 1.2497


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb5 (Functional)     │ (None, 15, 15, 2048)   │    28,513,527 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         5,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,505,000 (349.06 MB)

 Trainable params: 30,444,085 (116.13 MB)

 Non-trainable params: 172,743 (674.78 KB)

 Optimizer params: 60,888,172 (232.27 MB)

In [12]:
# base_model = EfficientNetB5(include_top=False, weights='imagenet', input_shape=(456, 456, 3))
# base_model.trainable = False  # Freeze the base model initially

# model = Sequential([
#     base_model,
#     GlobalAveragePooling2D(),
#     Dense(1024, activation='relu'),
#     Dense(len(label_map), activation='softmax')
# ])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


## Train the Model

In [13]:
# with strategy.scope():
#     history = model.fit(
#         train_generator,
#         epochs=5,  # Number of epochs to train for
#         validation_data=validation_generator,
#         verbose=1  # Shows a progress bar with updates after each batch is processed
#     )



## Fine Tune Model

In [14]:
#     base_model.trainable = True
#     model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

#     history_fine = model.fit(
#         train_generator,
#         epochs=5,
#         validation_data=validation_generator
#     )
